In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import math
import time

# Timing Decorater Function

In [2]:
def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"행렬 연산 {func.__name__} 실행 시간: {end_time - start_time:.4f}초")
        return result
    return wrapper


# Self-Head Attention (Scaled Dot Attention)

In [134]:
class selfAttention(nn.Module):
    # 데코레이터 사용해서 행렬 계산 시간 측정.
    @timing_decorator
    def __init__(self, embed_dim, attention_dim, dropout_rate=0.1):
        super().__init__()
        # embedding dim 입력을 받아서, attention dim 으로 변환
        self.embed_size = embed_dim
        self.attention_dim = attention_dim

        # Q, K, V 행렬 변환 레이어.
        self.W_q = nn.Linear(embed_dim, attention_dim)
        self.W_k = nn.Linear(embed_dim, attention_dim)
        self.W_v = nn.Linear(embed_dim, attention_dim)

        # 출력 프로젝션 (원래 차원으로 복원)
        self.out_proj = nn.Linear(attention_dim, embed_dim)

        # 드롭아웃 추가 (정규화 효과)
        self.dropout = nn.Dropout(dropout_rate)
    
    @timing_decorator
    def forward(self, x, mask=None):
        # Q, K, V 행렬 계산
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)

        # Q, K 행렬의 내적 계산 <- arguument (-2, -1) 은 tensor의 뒤 두 차원을 뒤집어서 계산 / batch 고려
        attention_score = Q @ K.transpose(-2, -1)
        # 내적 결과를 정규화
        attention_score = attention_score / math.sqrt(self.attention_dim)
        # 마스킹 지원 
        if mask is not None:
            attention_score = attention_score.masked_fill(mask == 0, -1e9)
        # 정규화된 내적 결과에 대한 소프트맥스 함수 적용, 행 적용.
        attention_score = self.dropout(F.softmax(attention_score, dim=-1))

        # 소프트맥스 결과와 V 행렬의 곱 계산
        attention_score = attention_score @ V
        # 원래 차원으로 복원
        output = self.out_proj(attention_score)
        return output


In [135]:
model = selfAttention(embed_dim=10000, attention_dim=100)
model(torch.randn(1, 10000, 10000)).shape

행렬 연산 __init__ 실행 시간: 0.0350초
행렬 연산 forward 실행 시간: 0.6955초


torch.Size([1, 10000, 10000])

# claude 3.7 최적화 버전

In [3]:
class selfAttention(nn.Module):
    @timing_decorator
    def __init__(self, embed_dim, attention_dim, dropout_rate=0.1):
        super().__init__()
        # embedding dim 입력을 받아서, attention dim 으로 변환
        self.embed_size = embed_dim
        self.attention_dim = attention_dim
        self.scale = math.sqrt(attention_dim)
        
        # 1. 행렬 연산 통합 - 하나의 선형 레이어로 Q, K, V 동시 계산
        # 기존에는 Q, K, V를 각각 접근하여 계산해, 메모리 접근이 비효율적. -> 하나의 선형 레이어로 계산
        # 큰 레이어로 계산하고 나누는 형식으로 사용.
        self.qkv_proj = nn.Linear(embed_dim, 3 * attention_dim)
        
        # 2. 출력 프로젝션 추가 (원래 차원으로 복원)
        self.out_proj = nn.Linear(attention_dim, embed_dim)
        
        # 3. 드롭아웃 추가 (정규화 효과)
        self.dropout = nn.Dropout(dropout_rate)
    
    @timing_decorator
    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.shape
        
        # 4. 통합된 QKV 계산 및 분리(chunk)
        qkv = self.qkv_proj(x).chunk(3, dim=-1)
        Q, K, V = qkv[0], qkv[1], qkv[2]
        
        # 5. 행렬 곱셈 최적화 (@ 연산자 사용)
        attention_score = (Q @ K.transpose(-2, -1)) / self.scale
        
        # 6. 마스킹 지원 추가
        if mask is not None:
            attention_score = attention_score.masked_fill(mask == 0, -1e9)
        
        # 7. 소프트맥스 및 드롭아웃
        attention_weights = self.dropout(F.softmax(attention_score, dim=-1))
        
        # 8. 가중치 적용 및 출력 프로젝션
        output = attention_weights @ V
        output = self.out_proj(output)
        
        return output

In [121]:
model = selfAttention(embed_dim=10000, attention_dim=100)
model(torch.randn(1, 10000, 10000), mask=None).shape

행렬 연산 __init__ 실행 시간: 0.0198초
행렬 연산 forward 실행 시간: 0.5833초


torch.Size([1, 10000, 10000])

# Multi-Head Attention

In [4]:
class multiHeadAttention(nn.Module):
    def __init__(self, embed_dim, head_num):
        super().__init__()
        
        self.embed_dim = embed_dim
        self.attention_dim = embed_dim // head_num
        self.head_num = head_num
        
        self.heads = nn.ModuleList([selfAttention(self.embed_dim, self.attention_dim) for _ in range(head_num)])

    @timing_decorator
    def forward(self, x, mask=None):
        return torch.cat([head(x, mask) for head in self.heads], dim=-1)
        


In [137]:
model = multiHeadAttention(embed_dim=10000, head_num=4)
model(torch.randn(1, 10000, 10000))

행렬 연산 __init__ 실행 시간: 0.3044초
행렬 연산 __init__ 실행 시간: 0.2930초
행렬 연산 __init__ 실행 시간: 0.2900초
행렬 연산 __init__ 실행 시간: 0.2926초
행렬 연산 forward 실행 시간: 2.5743초
행렬 연산 forward 실행 시간: 2.7165초
행렬 연산 forward 실행 시간: 2.7462초
행렬 연산 forward 실행 시간: 3.2253초
행렬 연산 forward 실행 시간: 11.9727초


tensor([[[-0.0217, -0.0096,  0.0127,  ...,  0.0042, -0.0054,  0.0151],
         [-0.0197, -0.0103,  0.0152,  ...,  0.0014, -0.0107,  0.0209],
         [-0.0182, -0.0118,  0.0141,  ..., -0.0017, -0.0094,  0.0192],
         ...,
         [-0.0201, -0.0081,  0.0132,  ..., -0.0009, -0.0114,  0.0178],
         [-0.0231, -0.0096,  0.0137,  ...,  0.0001, -0.0103,  0.0178],
         [-0.0188, -0.0105,  0.0091,  ...,  0.0005, -0.0121,  0.0194]]],
       grad_fn=<CatBackward0>)

# Claude 최적화

In [5]:
class multiHeadAttention(nn.Module):
    @timing_decorator
    def __init__(self, embed_dim, head_num, dropout_rate=0.1):
        super().__init__()
        
        self.embed_dim = embed_dim
        self.head_dim = embed_dim // head_num
        self.head_num = head_num
        
        # 1. 개별 selfAttention 모듈 대신 통합 프로젝션 사용
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        
        # 2. 출력 프로젝션 추가
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # 3. 드롭아웃 추가
        self.dropout = nn.Dropout(dropout_rate)
        
        # 4. 스케일링 팩터 미리 계산
        self.scale = self.head_dim ** -0.5
    
    @timing_decorator
    def forward(self, x, memory=None, mask=None):
        batch_size, seq_len, _ = x.shape
        
        # 5. 인코더-디코더 어텐션 지원
        # memory가 None이면 self-attention, 아니면 encoder-decoder attention
        if memory is None:
            # Self-Attention 모드
            memory = x
        
        # memory의 시퀀스 길이 가져오기
        _, mem_seq_len, _ = memory.shape
        
        # 6. 쿼리는 디코더 입력(x)에서, 키와 값은 인코더 출력(memory)에서 가져옴
        q = self.q_proj(x).view(batch_size, seq_len, self.head_num, self.head_dim)
        k = self.k_proj(memory).view(batch_size, mem_seq_len, self.head_num, self.head_dim)
        v = self.v_proj(memory).view(batch_size, mem_seq_len, self.head_num, self.head_dim)
        
        # 7. 차원 순서 변경으로 병렬 처리 최적화
        q = q.transpose(1, 2)  # [batch_size, head_num, seq_len, head_dim]
        k = k.transpose(1, 2)  # [batch_size, head_num, mem_seq_len, head_dim]
        v = v.transpose(1, 2)  # [batch_size, head_num, mem_seq_len, head_dim]
        
        # 8. 병렬 어텐션 계산
        # [batch_size, head_num, seq_len, mem_seq_len]
        attention_scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        
        # 9. 마스킹 지원 (인코더-디코더 어텐션용 마스크 처리)
        if mask is not None:
            # 마스크 형태에 따른 처리
            if mask.dim() == 3:  # [batch_size, seq_len, mem_seq_len]
                mask = mask.unsqueeze(1)  # [batch_size, 1, seq_len, mem_seq_len]
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
        
        # 10. 소프트맥스 및 드롭아웃
        attention_weights = self.dropout(F.softmax(attention_scores, dim=-1))
        
        # 11. 가중치 적용 및 차원 재구성
        # [batch_size, head_num, seq_len, head_dim]
        output = torch.matmul(attention_weights, v)
        
        # 12. 헤드 결합
        output = output.transpose(1, 2)  # [batch_size, seq_len, head_num, head_dim]
        output = output.reshape(batch_size, seq_len, self.embed_dim)
        
        # 13. 출력 프로젝션
        output = self.out_proj(output)
        
        return output

In [162]:
model = multiHeadAttention(embed_dim=10000, head_num=4)
model(torch.randn(1, 10000, 10000))

행렬 연산 __init__ 실행 시간: 1.2372초
행렬 연산 forward 실행 시간: 11.7844초


tensor([[[-0.0070,  0.0037,  0.0028,  ..., -0.0101,  0.0180,  0.0028],
         [-0.0075, -0.0023,  0.0051,  ..., -0.0098,  0.0193,  0.0026],
         [-0.0081,  0.0019, -0.0010,  ..., -0.0102,  0.0172,  0.0018],
         ...,
         [-0.0073,  0.0012,  0.0008,  ..., -0.0111,  0.0146, -0.0021],
         [-0.0103,  0.0013,  0.0017,  ..., -0.0128,  0.0145,  0.0039],
         [-0.0027,  0.0015,  0.0013,  ..., -0.0099,  0.0165,  0.0019]]],
       grad_fn=<ViewBackward0>)

# FFN

In [140]:
class FeedForward(nn.Module):
    @timing_decorator
    def __init__(self, embed_dim):
        super().__init__()
        self.embed_dim = embed_dim
        self.fc1 = nn.Linear(embed_dim, embed_dim)
        self.fc2 = nn.Linear(embed_dim, embed_dim)
    @timing_decorator
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

model = FeedForward(embed_dim=100)
model(torch.randn(100, 100))


행렬 연산 __init__ 실행 시간: 0.0061초
행렬 연산 forward 실행 시간: 0.0023초


tensor([[ 0.0949,  0.1345,  0.0263,  ..., -0.0280, -0.0293, -0.0613],
        [-0.0613, -0.0645,  0.1776,  ..., -0.3037, -0.2003, -0.0486],
        [ 0.3811,  0.3652,  0.1756,  ..., -0.2144,  0.1886, -0.0051],
        ...,
        [-0.1124,  0.0013, -0.2961,  ..., -0.1068, -0.1443, -0.0386],
        [-0.1595,  0.3505,  0.0045,  ..., -0.4028, -0.1005, -0.6073],
        [-0.2140,  0.0560,  0.1108,  ..., -0.0988, -0.3059, -0.3197]],
       grad_fn=<AddmmBackward0>)

In [8]:
class FeedForward(nn.Module):
    @timing_decorator
    def __init__(self, embed_dim, ff_dim=None, dropout_rate=0.1, activation='gelu'):
        super().__init__()
        
        # 1. 유연한 차원 설정 (일반적으로 ff_dim은 embed_dim의 4배)
        self.ff_dim = 4 * embed_dim if ff_dim is None else ff_dim
        
        # 2. 표준 Transformer 구조 적용
        self.fc1 = nn.Linear(embed_dim, self.ff_dim)
        self.fc2 = nn.Linear(self.ff_dim, embed_dim)
        
        # 3. 드롭아웃 추가
        self.dropout = nn.Dropout(dropout_rate)
        
        # 4. 활성화 함수 선택 지원
        if activation == 'relu':
            self.activation = F.relu
        elif activation == 'gelu':
            self.activation = F.gelu
        elif activation == 'silu' or activation == 'swish':
            self.activation = F.silu
        else:
            raise ValueError(f"지원하지 않는 활성화 함수: {activation}")
        
        # 5. 레이어 정규화 추가 (선택적)
        self.layer_norm = nn.LayerNorm(embed_dim)
        
        # 6. 가중치 초기화
        self._init_weights()
    
    def _init_weights(self):
        # Xavier/Glorot 초기화
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
    
    @timing_decorator
    def forward(self, x):
        # 7. 잔차 연결(residual connection) 적용
        residual = x
        
        # 8. 표준 피드포워드 네트워크 흐름
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        
        # 9. 잔차 연결 및 레이어 정규화
        x = residual + x
        x = self.layer_norm(x)
        
        return x

model = FeedForward(embed_dim=100)
model(torch.randn(100, 100))

행렬 연산 __init__ 실행 시간: 0.0012초
행렬 연산 forward 실행 시간: 0.0011초


tensor([[-0.9015, -0.8391,  1.3860,  ...,  1.7770,  1.1994, -0.8444],
        [-0.1553, -1.3756,  1.6660,  ...,  1.5024,  0.5604,  0.6539],
        [ 0.9111,  0.3385,  0.0722,  ..., -0.1994, -2.1402,  0.9485],
        ...,
        [-0.1498, -0.1131,  0.8218,  ..., -0.7722,  0.3327, -1.3569],
        [-0.6441,  0.4015,  0.8817,  ...,  0.2852,  0.1499, -0.5180],
        [ 1.6629, -0.4979, -0.5406,  ...,  1.0214,  0.3287, -0.1590]],
       grad_fn=<NativeLayerNormBackward0>)

# Positional Encoding

In [9]:
class PositionalEncoding(nn.Module):
    @timing_decorator
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        self.embed_dim = embed_dim
        self.max_len = max_len
        
        self.pe = torch.zeros(max_len, embed_dim)
        for pos in range(max_len):
            for i in range(embed_dim):
                if i % 2 == 0:
                    self.pe[pos, i] = math.sin(pos / (10000 ** (i/embed_dim)))
                else:
                    self.pe[pos, i] = math.cos(pos / (10000 ** ((i-1)/embed_dim)))
    def forward(self, x):
        return x + self.pe[:x.size(1), :]
    

In [10]:
embed_dim = 1000
model = PositionalEncoding(embed_dim=embed_dim)
model(torch.randn(embed_dim, embed_dim))

행렬 연산 __init__ 실행 시간: 12.2531초


tensor([[-1.2092,  1.6836,  1.2086,  ...,  1.9223,  0.7860,  0.9179],
        [ 0.5510,  0.6314,  1.1510,  ...,  0.7919,  2.3845,  1.1229],
        [ 0.7000, -0.8367,  1.3397,  ...,  0.7099,  1.0221,  1.6009],
        ...,
        [-1.1488,  0.5710, -0.4910,  ...,  0.6921,  0.0192,  3.6388],
        [-0.1267,  0.9919,  0.8790,  ...,  0.5224,  1.4226,  0.5621],
        [-0.0551,  0.3650,  1.0389,  ...,  0.7748,  0.3693,  1.4311]])

In [11]:
class PositionalEncoding(nn.Module):
    @timing_decorator
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        
        # 2. 벡터화 연산으로 변경 (중첩 for문 제거)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        
        pe = torch.zeros(max_len, embed_dim)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # 3. 등록된 버퍼로 변환 (모델 저장 시 함께 저장됨)
        self.register_buffer('pe', pe.unsqueeze(0))
        
        # 4. 임베딩 차원 저장 (디버깅 및 문서화 목적)
        self.embed_dim = embed_dim
    
    def forward(self, x):
        # 5. 입력 시퀀스 길이에 맞게 위치 인코딩 잘라서 사용
        # x 형태: (batch_size, seq_len, embed_dim)
        return x + self.pe[:, :x.size(1), :]

In [12]:
embed_dim = 1000
model = PositionalEncoding(embed_dim=embed_dim)
model(torch.randn(embed_dim, embed_dim))

행렬 연산 __init__ 실행 시간: 0.0083초


tensor([[[ 0.5656,  1.2081,  0.7978,  ...,  0.6967,  1.6393, -0.6661],
         [ 1.4990, -0.7042,  1.1031,  ..., -0.1064, -0.4657,  0.9256],
         [ 1.5408,  0.0763,  0.7209,  ...,  1.8202,  1.1380,  1.8009],
         ...,
         [-1.1181,  0.2585, -0.3091,  ..., -1.1144, -0.6131, -0.1160],
         [-2.6969,  1.8758, -0.0774,  ...,  0.4491,  0.7902,  0.9058],
         [-0.4565,  2.2082,  0.9516,  ..., -0.4351,  0.8622,  0.0206]]])

# Add & Norm

In [13]:
class EncoderLayer(nn.Module):
    @timing_decorator
    def __init__(self, embed_dim, head_num, ff_dim=None, dropout_rate=0.1):
        super().__init__()
        self.self_attn = multiHeadAttention(embed_dim, head_num, dropout_rate)
        self.ffn = FeedForward(embed_dim, ff_dim, dropout_rate)
        
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        
    @timing_decorator
    def forward(self, x, mask=None):
        x = self.norm1(x + self.self_attn(x, mask))
        x = self.norm2(x + self.ffn(x))
        return x

In [14]:
model = EncoderLayer(embed_dim=100, head_num=4)
model(torch.randn(1, 100, 100))

행렬 연산 __init__ 실행 시간: 0.0008초
행렬 연산 __init__ 실행 시간: 0.0007초
행렬 연산 __init__ 실행 시간: 0.0016초
행렬 연산 forward 실행 시간: 0.0009초
행렬 연산 forward 실행 시간: 0.0007초
행렬 연산 forward 실행 시간: 0.0018초


tensor([[[ 2.1076,  0.9659, -2.2989,  ...,  0.9238,  0.9898, -1.1221],
         [-1.3739,  0.8757, -0.0176,  ..., -0.7255, -2.1582, -1.6540],
         [ 0.5585,  0.8772, -0.4673,  ..., -1.0803,  0.6039, -0.8235],
         ...,
         [-0.5640,  1.8456,  0.6362,  ..., -0.2410,  0.5605,  0.2439],
         [-0.1371,  1.7360, -1.7944,  ..., -0.8012,  1.3335, -1.3292],
         [ 0.6319, -0.5479, -0.6247,  ..., -0.8504, -0.2272, -0.9490]]],
       grad_fn=<NativeLayerNormBackward0>)

In [15]:
class DecoderLayer(nn.Module):
    @timing_decorator
    def __init__(self, embed_dim, head_num, ff_dim=None, dropout_rate=0.1):
        super().__init__()
        self.self_attn = multiHeadAttention(embed_dim, head_num, dropout_rate)
        self.cross_attn = multiHeadAttention(embed_dim, head_num, dropout_rate)
        self.ffn = FeedForward(embed_dim, ff_dim, dropout_rate)
        
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        
    @timing_decorator
    def forward(self, x, memory, mask=None):
        x = self.norm1(x + self.self_attn(x, mask))
        x = self.norm2(x + self.cross_attn(x, memory, mask))
        x = self.norm3(x + self.ffn(x))
        return x


In [16]:
model = DecoderLayer(embed_dim=100, head_num=4)
model(x = torch.randn(1, 100, 100), memory = torch.randn(1, 100, 100))

행렬 연산 __init__ 실행 시간: 0.0009초
행렬 연산 __init__ 실행 시간: 0.0005초
행렬 연산 __init__ 실행 시간: 0.0006초
행렬 연산 __init__ 실행 시간: 0.0024초
행렬 연산 forward 실행 시간: 0.0008초
행렬 연산 forward 실행 시간: 0.0008초
행렬 연산 forward 실행 시간: 0.0007초
행렬 연산 forward 실행 시간: 0.0028초


tensor([[[ 0.1695,  0.5823,  0.6928,  ..., -1.6887, -1.8353, -0.3903],
         [ 0.7396, -0.2839,  0.7600,  ..., -0.3781,  0.1145, -1.4993],
         [-0.0114,  0.6360,  1.4768,  ..., -0.3988, -1.0712,  1.6964],
         ...,
         [ 0.7200,  0.2242,  1.1404,  ..., -0.0574,  1.0430,  0.4501],
         [-1.6600, -0.7676, -0.8377,  ...,  0.0948, -1.8584, -0.8328],
         [ 0.4964,  1.1080,  0.4698,  ..., -1.6858,  1.0066, -0.3532]]],
       grad_fn=<NativeLayerNormBackward0>)

In [17]:
class Transformer(nn.Module):
    @timing_decorator
    def __init__(self, embed_dim, head_num, ff_dim=None, dropout_rate=0.1):
        super().__init__()
        self.encoder = EncoderLayer(embed_dim, head_num, ff_dim, dropout_rate)
        self.decoder = DecoderLayer(embed_dim, head_num, ff_dim, dropout_rate)
        self.linear = nn.Linear(embed_dim, embed_dim)

    @timing_decorator
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        src = self.encoder(src, src_mask)
        tgt = self.decoder(tgt, src, tgt_mask)        
        return self.linear(tgt)


In [18]:
embed_dim = 10000
head_num = 8
model = Transformer(embed_dim=embed_dim, head_num=head_num)
model(torch.randn(1, 100, embed_dim), torch.randn(1, 100, embed_dim))

행렬 연산 __init__ 실행 시간: 1.1658초
행렬 연산 __init__ 실행 시간: 4.3232초
행렬 연산 __init__ 실행 시간: 5.4894초
행렬 연산 __init__ 실행 시간: 1.1191초
행렬 연산 __init__ 실행 시간: 1.1293초
행렬 연산 __init__ 실행 시간: 4.5027초
행렬 연산 __init__ 실행 시간: 6.7520초
행렬 연산 __init__ 실행 시간: 12.5215초
행렬 연산 forward 실행 시간: 0.1201초
행렬 연산 forward 실행 시간: 0.2046초
행렬 연산 forward 실행 시간: 0.3272초
행렬 연산 forward 실행 시간: 0.1172초
행렬 연산 forward 실행 시간: 0.1161초
행렬 연산 forward 실행 시간: 0.2542초
행렬 연산 forward 실행 시간: 0.4906초
행렬 연산 forward 실행 시간: 0.8446초


tensor([[[-0.4427, -1.5164,  0.2352,  ...,  0.6473,  0.7047, -0.4855],
         [ 0.1503, -1.1656,  1.5674,  ..., -1.0098,  0.2365, -0.1964],
         [-0.8602,  0.2901,  0.6377,  ...,  0.0171,  0.6700,  0.9817],
         ...,
         [-0.5442, -0.5437,  0.2300,  ..., -0.0323,  0.4300, -0.2441],
         [ 0.1137,  0.2401, -0.7411,  ..., -0.4218,  0.5344,  1.0317],
         [-0.3432,  1.3753, -0.5390,  ..., -1.5395,  0.4840, -0.4992]]],
       grad_fn=<ViewBackward0>)